In [3]:
# Import libraries

import pandas as pd
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [29]:
#configuration

path = 'dataset/'
products = 'products-data*.tsv'
reviews = 'reviews-*.tsv'
products_column_names = ['id', 'category', 'product title']
review_column_names=['id', 'rating', 'review_text']






In [25]:
def data_loader(path, name_of_file , column_names):
    current_items_list = []
    data_files = Path(path).glob(name_of_file)
    for file in data_files:
        current_items_list.append(pd.read_csv(file, sep = '\t', header = None, names = column_names))
    return pd.concat(current_items_list)

In [26]:
df_product = data_loader(path,products,products_column_names)

In [30]:
df_product = data_loader(path,reviews,review_column_names)

In [31]:
df_product

,id,rating,review_text
0,daa54754-af9c-41c0-b542-fe5eabc5919c,5,Great!
1,ab602aca-9bad-4aa9-bd42-6ce24cdf8680,5,It fit perfectly on my new<br />Victorinox Fib...
2,f98cd8d3-d6fe-4ee3-8c9f-a18c1690f7a6,5,Excellent pan - my daughter just made madelein...
3,396f73e3-e3ef-4c93-a4b7-e8b04c6a2223,3,not the good one but not bad
4,329ed5e3-8b47-4e78-961b-18b89e7da808,1,This is a rip off. It only comes with this nec...
...,...,...,...
249,def6de43-649a-47c8-9b79-2dd94f21689d,5,Great product delivered on time. it is a wonde...
250,a2b774d3-5a75-47bb-91d8-59c8e3769f51,2,Snap broke before I even wore it
251,0e6c70dd-44d6-40e5-b635-360302acc275,5,"Love the BLACK color, and the setting is beaut..."
252,1df86592-7c6b-4194-ae09-7a2ea30093a0,5,Earrings are lovely. I had a question about th...
